In [4]:
import numpy as np
import warnings
import inspect
from euclid import *
# import carla

from collections.abc import MutableMapping
from cvxopt import matrix
import sys
import os

sys.path.append(os.path.dirname(os.path.abspath('')) +
                "../../")

try:
    from cbf.obstacles import Ellipse2D, ObstacleList2D, Obstacle2DBase, BoundingBox
    from cbf.geometry import Rotation
except:
    raise


In [4]:
ellipse2d1 = Ellipse2D(2,3)
ellipse2d2 = Ellipse2D(3,4)

In [5]:
obs2dlist = ObstacleList2D({"id1":ellipse2d1, "id2":ellipse2d2})

In [6]:
obs2dlist

ObstacleList2D({'id1': Ellipse2D(a = 2, b = 3, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
, 'id2': Ellipse2D(a = 3, b = 4, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
})

In [7]:
p = Vector2(2,0)

In [8]:
print(obs2dlist.f(p))

[ 0.00e+00]
[-5.56e-01]



In [9]:
ellipse2d3 = Ellipse2D(5,5)

In [10]:
obs2dlist.update(id3 = ellipse2d3)

In [11]:
obs2dlist

ObstacleList2D({'id1': Ellipse2D(a = 2, b = 3, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
, 'id2': Ellipse2D(a = 3, b = 4, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
, 'id3': Ellipse2D(a = 5, b = 5, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
})

In [12]:
obs2dlist['id4'] = ellipse2d3

In [13]:
obs2dlist

ObstacleList2D({'id1': Ellipse2D(a = 2, b = 3, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
, 'id2': Ellipse2D(a = 3, b = 4, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
, 'id3': Ellipse2D(a = 5, b = 5, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
, 'id4': Ellipse2D(a = 5, b = 5, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
})

In [14]:
obs2dlist['id4'].update(a=5, b=6)

In [15]:
obs2dlist

ObstacleList2D({'id1': Ellipse2D(a = 2, b = 3, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
, 'id2': Ellipse2D(a = 3, b = 4, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
, 'id3': Ellipse2D(a = 5, b = 6, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
, 'id4': Ellipse2D(a = 5, b = 6, center = Vector2(0.00, 0.00), theta = 0, buffer = 0, buffer_applied: True )
})

In [28]:
def test_func(*args):
    for arg in args:
        print(arg)
    return args

In [29]:
a, b, c, d, e = test_func(1,2,3,45, 'hey')

1
2
3
45
hey


In [32]:
a = Point3()

In [33]:
a

Point3(0.00, 0.00, 0.00)

In [40]:
isinstance(ellipse2d1, Ellipse2D)

True

In [41]:
class Shape:
    pass

class Rectangle(Shape):
    def __init__(self, length, width):
        self.length = length
        self.width = width
        self.area = length * width
    
    def get_area(self):
        return self.length * self.width
        
class Square(Rectangle):
    def __init__(self,length):
        Rectangle.__init__(self,length,length)

In [42]:
test_sq = Square(2)

In [45]:
isinstance(test_sq, Square)

True

In [46]:
test_rq = Rectangle(2,3)

In [47]:
isinstance(test_rq, Shape)

True

In [48]:
isinstance(test_rq, Square)

False

In [5]:
bbox1 = BoundingBox(extent=Vector3(1,1,1), location=Vector3(4,5,2), rotation=Rotation(0, 0, np.pi))
bbox2 = BoundingBox(extent=Vector3(1,1.5,1.5), location=Vector3(5,3,2.5), rotation=Rotation(0, 0, np.pi))

In [6]:
bbox_dict = {
    'id1':bbox1,
    'id2':bbox2
}

In [8]:
obs_list = ObstacleList2D()

In [9]:
obs_list.update_by_bounding_box(bbox_dict=bbox_dict)

In [10]:
obs_list

ObstacleList2D({'id1': Ellipse2D(a = 1.5, b = 1.5, center = Vector2(4.00, 5.00), theta = 3.141592653589793, buffer = 0.5, buffer_applied: True )
, 'id2': Ellipse2D(a = 1.5, b = 2.0, center = Vector2(5.00, 3.00), theta = 3.141592653589793, buffer = 0.5, buffer_applied: True )
})

In [12]:
bbox2 = BoundingBox(extent=Vector3(1,1.5,1.5), location=Vector3(6,6,2.5), rotation=Rotation(0, 0, np.pi/2))
bbox_dict.update({'id2':bbox2})

In [13]:
obs_list.update_by_bounding_box(bbox_dict=bbox_dict)

In [14]:
obs_list

ObstacleList2D({'id1': Ellipse2D(a = 1, b = 1, center = Vector2(4.00, 5.00), theta = 3.141592653589793, buffer = 0.5, buffer_applied: True )
, 'id2': Ellipse2D(a = 1, b = 1.5, center = Vector2(6.00, 6.00), theta = 1.5707963267948966, buffer = 0.5, buffer_applied: True )
})